<a href="https://colab.research.google.com/github/EgorDudyrev/OntologyPailleur/blob/main/FCA_From_Scratch__SymbolicKD_at_IDMC_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbolic Knowledge Discovery with FCA, TD at IDMC 2026

This Jupyter notebook provides the first hands-on expirience with Formal Concept Analysis in Python.
It is designed in a way so that you can practice converting the math of FCA into Python code, so that you can evaluate the efficiency of the algorithms you've studied (e.g. NextClosure), and so that you can see how FCA output can look on the real data.

The notebook is split into 6 (+1) parts:

0. Load the data
1. Basic FCA operations
2. Mining concepts in a smart way (i.e. NextClosure),
3. Mining implications in a smart way (through Carpathia-G algorithm),
4. Mining association rules (with Luxenburger basis),
5. Trying out the real data (the Titanic dataset),
6. Pushing the concepts to GraphRAG (a small hint on the project).

But keep calm, *you are only asked to write some code in Parts 1, 2, and 3*: the other sections serve demonstration purposes only.

## Part 0. Load the data

In [ ]:
!pip install --quiet caspailleur

Download one of the classical FCA datasets available at the Formal Context Repository: https://fcarepository.org/

In [ ]:
import pandas as pd
from caspailleur.io import from_fca_repo
df, metadata = from_fca_repo('newzealand_en.cxt')
df

To make the code more structured and more similar to the mathematical definitions, we represent the dataset with this simple `FormalContext` class:

In [ ]:
from typing import NamedTuple

class FormalContext(NamedTuple):
  objects: set[str]
  attributes: set[str]
  relations: set[tuple[str, str]]

In [ ]:
def pandas_to_sets(df) -> FormalContext:
  objects = set(df.index)
  attributes = set(df.columns)
  relations = {(obj, attr) for obj in objects for attr in attributes if df.at[obj, attr]}
  return FormalContext(objects, attributes, relations)

In [ ]:
context = pandas_to_sets(df)
print(f"{context.objects=}")
print(f"{context.attributes=}")
print(f"{context.relations=}")

## Part 1. Basic operations

This section introduces two main operations in FCA: the extent and the intent (both are often represented with $\cdot'$).

With these two basic operations (and some minor adjustments), we can already mine concepts and implications. Although such bruteforce algorithms can take a lot of time of the big data.

Recall that a Formal Context is a triplet $(G, M, I)$ of the set of objects $G$, the set of attributes $M$, and the incidence relation $I \subseteq G \times M$ between objects and attributes.

Extent of description $B$ is the (maximal) set of objects described by $B$:
$$\mathtt{extent}(B) = B' = \{g \in G \mid \forall m \in M\ (g, m) \in I \}$$

In [ ]:
def extent(description: set[str], context: FormalContext) -> set[str]:
  # TODO: Implement the extent operation
  # HINT: Pythonic set-comprehension looks A LOT like the mathematical set-builder notation above
  return ...

extent({'Jet Boating', 'Wildwater Rafting'}, context)

Intent of a subset of objects $A$ is the (maximal) set of attributes that describe $A$:
$$\mathtt{intent}(A) = A' = \{m \in M \mid \forall g \in A, (g,m) \in I \}$$

In [ ]:
def intent(objects: set[str], context: FormalContext) -> set[str]:
  # TODO: Implement the intent operation
  return ...

intent({'Queenstown', 'Wanaka'}, context)

Let us import the `powerset` function from `caspailleur` package in order to iterate through the powerset of a set (i.e. iterate through all the subsets of some given set)

In [ ]:
from caspailleur.base_functions import powerset

list(powerset({'a', 'b', 'c'}))

Here we implement the bruteforce algorithm for mining formal concepts: it goes through all the (exponential amount of) possible descriptions, and computes their extents and intents.

In [ ]:
class FormalConcept(NamedTuple):
  extent: set[str]  # all objects covered by the concept
  intent: set[str]  # all attributes covered by the concept
  support: float = None  # the number of objects in the extent (optional)

  def __hash__(self) -> int:
    return hash((frozenset(self.extent), frozenset(self.intent)))

In [ ]:
def mine_concepts_bruteforce(context: FormalContext) -> set[FormalConcept]:
  concepts = set()
  for description in powerset(context.attributes):
    # ToDo: Finish the Bruteforce algorithm for mining concepts.
    # Compute the extent and the intent of a given description and form them into a FormalConcept
    extent_ = ...
    intent_ = ...
    concepts.add(FormalConcept(extent_, intent_, len(extent_)/len(context.objects)))
  return concepts

In [ ]:
%%time
concepts = mine_concepts_bruteforce(context)
pd.DataFrame(concepts).sort_values('support', ascending=False)

Bruteforce generation of implications. Again, we pass through every subset of attributes and use it to make an implication "subset_of_attributes => its_closure". And then we filter out some of the "reduntant" implications to get the Proper Premise basis of implications.

In [ ]:
class Implication(NamedTuple):
  premise: set[str]
  conclusion: set[str]
  support: float = None

  def __hash__(self) -> int:
    return hash((frozenset(self.premise), frozenset(self.conclusion)))

In [ ]:
def mine_implications_bruteforce(context: FormalContext) -> set[Implication]:
  implications = set()
  for description in powerset(context.attributes):
    # ToDo: Compute the `extent_` and the `intent_` of the description to form an implication "description => intent_"
    extent_ = ...
    intent_ = ...

    # And now we filter some "obvious" and "reduntant" implications,
    # so that the final set of implications would make a Proper Premise basis (aka Canonical Direct basis)

    # if implication is obvious: i.e. description => description, then we do not want to see it
    if description == intent_:
      continue

    # if implication can be expressed by combining smaller implications, then we do not want to see it
    subdescriptions = {description - {attr} for attr in description}
    subintent = set(description)
    for subdescription in subdescriptions:
      subintent |= intent(extent(subdescription, context), context)
    if subintent == intent_:
      continue

    # we remove duplicating attributes from the conclusion, so that the implication would look more concise
    conclusion = intent_ - description

    support = len(extent_)/ len(context.objects)
    implications.add(Implication(description, conclusion, support))
  return implications

In [ ]:
%%time
implications = mine_implications_bruteforce(context)
pd.DataFrame(implications).sort_values('support', ascending=False)

## Part 2. Mining concepts in a smart way

The bruteforce way of iterating through all subsets of attributes might be very expensive when the data is big. So it is better to use some advanced algorithms. For example, NextClosure.

In [ ]:
def is_lectically_smaller(description_left: set[str], description_right: set[str], index: int, attributes_list: list[str]) -> bool:
  # ToDo: Implement lectic order comparison for the next closure algorithm
  # The formula can be found on Slide 63 of the class where "description_left" is reffered to as "A" and "description_right" is reffered to as "B".
  # Note that the "index" here is a number, and not an attributes (as on the slides)
  ...
  return ...


def next_closure(intent_: set[str], attributes_list: list[str], context: FormalContext) -> frozenset[str]:
  # ToDo: Implement the NextClosure algorithm from Slide 65 of the class
  for i in reversed(range(len(attributes_list))):
    ...
  return context.attributes

In [ ]:
def all_closure(context: FormalContext) -> set[FormalConcept]:
  # AllClosure function. It is quite simple, so there is nothing to implement.
  concepts = set()

  attributes_list = sorted(context.attributes)
  intent_ = intent(extent(set(), context), context)
  while True:
    extent_ = extent(intent_, context)
    concepts.add(FormalConcept(extent_, intent_, len(extent_)/len(context.objects)))

    if intent_ == context.attributes:
      break

    intent_ = next_closure(intent_, attributes_list, context)

  return concepts

In [ ]:
%%time
concepts = all_closure(context)
pd.DataFrame(concepts).sort_values('support', ascending=False)

Make sure that the bruteforce and the smart way give the same output

In [ ]:
assert all_closure(context) == mine_concepts_bruteforce(context)

In [ ]:
%timeit mine_concepts_bruteforce(context)
%timeit all_closure(context)

## Part 3. Mining implications in a smart(er) way

In this Part we mine Proper Premise basis of implications in a smarter way.

Recall that a **proper premise** is defined as follows:
$$ P \subseteq M \text{ is a proper premise} \iff P \cup \bigcup_{m \in P} (P \setminus \{m\})'' \neq P''.$$

The mathematical definition implies an important property:
$$P \cup \bigcup_{m \in P} (P \setminus \{m\})'' \neq P'' \implies (P \setminus \{m\})'' \neq P'', \forall m \in P.$$
To put it into words:
$$P \text{ is a proper premise} \implies P \text{ is a minimal generator}.$$

And **minimal generator** is a description, such that its every subset of attributes describes more objects. So, it is impossible to make a minimal generator smaller and still describe the same objects.

To conclude, one way to mine proper premises would be to 1) mine all the minimal generators, 2) filter out minimal generators that are not proper premises.

Below is one of a simple-yet-efficient algorithm for mining minimal generators called Carpathia-G.

The algorithm uses one property of a minimal generator: $$\text{every subset of a minimal generator is a minimal generator}.$$

That is, if there is some description s.t. one of its subsets is not a minimal generator, then the description itself is not a minimal generator.

In [1]:
def carpathia_g(context: FormalContext) -> list[set[str]]:
  mingens: set[frozenset[str]] = set()
  attributes_list = list(context.attributes)

  # The algorithm goes through various subsets of attributes, starting from the smallest one (the empty subset) and incrementally going to more and more complex descriptions
  queue = [frozenset()]
  while queue:
    description = queue.pop(0)
    subdescriptions = {description - {attr} for attr in description}
    # ToDo: Implement Test #1: if some subdescriptions are not in `mingens` set, the `description` is not a minimal generator
    if ...:
      continue  # not a minimal generator, so go to another description in the queue

    # ToDo: Implement Test #2: if some subdescription is describes the same extent, then `description` is not a minimal generator
    extent_ = extent(description, context)
    if ...:
      continue  # not a minimal generator, so go to another description in the queue

    # All tests passed, so the description is a minimal generator
    mingens.add(description)

    # If the two tests were successfull, then we can try to add one more attributes to the description
    max_idx = max(attributes_list.index(attr) for attr in description) if description else -1
    next_descriptions = [description | {m} for m in attributes_list[max_idx+1:]]
    queue.extend(next_descriptions)

  return [frozenset(mingen) for mingen in mingens]

NameError: name 'FormalContext' is not defined

We know how to iterate minimal generators (using Carpathia-G), now we just should identify the minimal generators that are also proper premises. You can do that by following the definition of the proper premise above.

In [2]:
def is_proper_premise(min_generator: frozenset[str], context: FormalContext) -> bool:
  # ToDo: Implement the definition of a proper premise (the first formula in Part 3) to test if a `min_generator` is a proper premise,
  ...

NameError: name 'FormalContext' is not defined

In [ ]:
def mine_implications_smart(context: FormalContext) -> set[Implication]:
  implications: set[Implication] = set()

  attributes_list = list(context.attributes)

  # the algorithm is simple: go through every minimal generator, if it is a proper premise, then make it an Implication
  for mingen in carpathia_g(context):
    if is_proper_premise(mingen, context):
      extent_ = extent(mingen, context)
      conclusion = intent(extent_, context) - mingen
      support = len(extent_) / len(context.objects)
      implications.add(Implication(set(mingen), conclusion, support))

  return implications

Make sure that the bruteforce and the smart way give the smae output

In [ ]:
assert mine_implications_smart(context) == mine_implications_bruteforce(context)

In [ ]:
%timeit mine_implications_bruteforce(context)
%timeit mine_implications_smart(context)

## Part 4. Association rules

Implications can often be too "strict" to use them on real data. One of the solutions is to mine **association rules** that are just like the implications rules but with varying confidence.

In [ ]:
class AssociationRule(NamedTuple):
  premise: set[str]
  conclusion: set[str]
  confidence: float = None
  support: float = None

  def __hash__(self):
    return hash((frozenset(self.premise), frozenset(self.conclusion)))

One of the classical way to compute the basis of association rules is to use so-called Luxenburger basis. It finds all pairs of neighbouring concepts in a concept lattice and makes an association rule that the intent of the greater concept associates to the intent of the smaller concept.

In [ ]:
def luxenburger_basis(context: FormalContext) -> set[AssociationRule]:
  concepts = all_closure(context)

  associations = set()

  for concept in concepts:
    subconcepts = {other for other in concepts if other.extent < concept.extent}
    top_subconcepts = {
        subconcept for subconcept in subconcepts
        if not any(subconcept.extent < higher_sub.extent for higher_sub in subconcepts)
    }

    for subconcept in top_subconcepts:
      confidence = len(subconcept.extent)/len(concept.extent) if subconcept.extent else 1
      support = len(subconcept.extent) / len(context.objects)

      conclusion = subconcept.intent - concept.intent
      associations.add(AssociationRule(concept.intent, conclusion, confidence, support))
  return associations

In [ ]:
pd.DataFrame(luxenburger_basis(context))

## Part 5. The Real data

Let us try to run FCA algorithms on some real data, e.g. the Titanic one.
All the code here is already written but you are invited to generate new binary attributes on this complex data.

There are two important questions: 1) what are some interesting dependancies on the Titanic data, and 2) at what point the running time of FCA algorithms will explode exponentially.

In [ ]:
titanic_df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv', index_col=0)
titanic_df.index = 'Passenger '+titanic_df.index.astype(str)
titanic_df.head()

In [ ]:
# Here are some of possible binary attributes. You can always propose your own attributes.
titanic_bin = {
    'Survived': titanic_df['Survived'] == 1,
    'FirstClass': titanic_df['Pclass'] == 1,
    'French': titanic_df['Embarked'] == 'C',  # 'C' means 'Cherbourg'
    'Adult': titanic_df['Age'] >= 18,
    'Child': titanic_df['Age'] < 18,
    'Male': titanic_df['Sex'] == 'male',
    'Female': titanic_df['Sex'] == 'female',
}
titanic_bin = pd.DataFrame(titanic_bin)
titanic_bin.head()

In [ ]:
titanic_context = pandas_to_sets(titanic_bin)

In [ ]:
assert mine_concepts_bruteforce(titanic_context) == all_closure(titanic_context)
%timeit mine_concepts_bruteforce(titanic_context)
%timeit all_closure(titanic_context)

Note that sometimes advanced algorithms can work slower than the bruteforce ones: that happens when the data is too simple.

Another way to make the algorithms faster is by using appropriate data structures. In particular, binary datasets can be represented as a list of bitmasks (or `bitarray`s). This way, the algorithms might work thousand times faster.

In [ ]:
concepts = all_closure(titanic_context)
pd.DataFrame(concepts).sort_values('support', ascending=False)

In [ ]:
assert mine_implications_smart(titanic_context) == mine_implications_bruteforce(titanic_context)
%timeit mine_implications_bruteforce(titanic_context)
%timeit mine_implications_smart(titanic_context)

In [ ]:
implications = mine_implications_smart(titanic_context)
pd.DataFrame(implications).sort_values('support', ascending=False)

In [ ]:
associations = luxenburger_basis(titanic_context)
association_df = pd.DataFrame(associations).sort_values(['support', 'confidence'], ascending=False).reset_index(drop=True)
association_df = association_df[(association_df['confidence']>0.5)&(association_df['support']>=0.1)]
association_df

## Part 6. Integration with GraphRAG and Ontologies

And here is some hint on what will happen during the second half of the course and during the project work.

Except that we will learn how to find concepts, implications, and associations on complex data (i.e. numerical, categorical, graphs, etc.), so the binarisation step will become obsolete.

### Part 6.1. Create ontology

In [ ]:
!pip install --quiet git+https://github.com/smartFCA/OntologyPailleur

We should change the concepts representation a little bit. In `OntologyPailleur` package every formal concept is a triple of (extent, intent, minimal_generators).


In [ ]:
mingens_per_closure = {frozenset(implication.premise | implication.conclusion): [] for implication in implications}
for implication in implications:
  closure = frozenset(implication.premise | implication.conclusion)
  mingens_per_closure[closure].append(implication.premise)

In [ ]:
from ontopailleur import formal_ontology_constructor as foc
concepts_to_save = [foc.FormalConcept(extent(intent, titanic_context), intent, mingens) for intent, mingens in mingens_per_closure.items()]

We use `rdflib` to create knowledge graphs in Turtle format (.ttl) within Python.

Every node of a knowledge graph (including the ones representing objects $G$ and attributes $M$), should be represented with some `rdflib.URIRef(node_name)`. The name of the node should start with a prefix (that can be done via `rdflib.Namespace`), and should contain neither empty spaces nor special symbols.

The code below compute automatic URIRef's for each object and attribute, but you might need to change the code if you have some particular names of objects and attributes.

In [ ]:
import rdflib
NSpace = rdflib.Namespace('http://idmc.univ-lorraine.fr/td_fca#')
obj_to_refs = {obj: rdflib.URIRef(NSpace[obj.replace(' ', '_').capitalize()]) for obj in titanic_context.objects}
attr_to_refs = {attr: rdflib.URIRef(NSpace[attr.replace(' ', '_').capitalize()]) for attr in titanic_context.attributes}

attr_to_refs

In [ ]:
kgraph = foc.construct_ontology(NSpace, prefix='tdfca', objects_to_refs=obj_to_refs, attributes_to_refs=attr_to_refs, incidence_relation=titanic_context.relations, concepts=concepts_to_save)
print("First 5 RDF triplets of the knowledge graph")
list(kgraph.triples((None, None, None)))[:5]

In [ ]:
kgraph.serialize('kgraph.ttl')

You can download the saved graph and open it in Protégé system as an ontology.

### Part 6.2 Running GraphRAG

Now we want to put the obtained Knowledge Graph into a GraphRAG pipeline. To do so, we should download some more packages and neural networks.

In [ ]:
!pip install --quiet networkx matplotlib torch transformers sentence-transformers lmdb -q
!pip install --quiet tf_keras

In [ ]:
from ontopailleur import graphrag

EMBEDDING_MODEL_NAME: str = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL_NAME: str = 'Qwen/Qwen2-1.5B-Instruct'  # 'Qwen/Qwen2-0.5B-Instruct'
LMDB_PATH: str = './embeddings_local'
GRAPH_PATH = 'kgraph.ttl'

In [ ]:
embedding_model = graphrag.compute_embeddings(GRAPH_PATH, EMBEDDING_MODEL_NAME, LMDB_PATH)

In [ ]:
grag = graphrag.GraphRAG(GRAPH_PATH, LMDB_PATH, embedding_model)

In [ ]:
tokenizer, model = graphrag.initialise_llm(LLM_MODEL_NAME)

Now we can ask use GraphRAG system for asking questions about the data.

In [ ]:
graphrag.graphrag_query(grag, tokenizer, model, 'What describes Passenger 520?', hops=1)